# Neural Network Regression - No Stratification

# Project 3 - Predicting the Age of Abalone

In [1]:
# Dependancies

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

## Data Preprocessing

In [2]:
# Read in unscaled version of the dataset
unscaled_df = pd.read_csv('./Dataset/abalone_unscaled_data.csv')
unscaled_df.head()

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Age
0,M,91,73,19,102.8,44.9,20.2,30.0,15,16.5
1,M,70,53,18,45.1,19.9,9.7,14.0,7,8.5
2,F,106,84,27,135.4,51.3,28.3,42.0,9,10.5
3,M,88,73,25,103.2,43.1,22.8,31.0,10,11.5
4,I,66,51,16,41.0,17.9,7.9,11.0,7,8.5


In [3]:
# Label Encode the categorical "Sex" column
# Note: Female => 0
#       Infant => 1
#       Male   => 2
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
unscaled_df['Sex_LabelEncoded'] = le.fit_transform(unscaled_df['Sex'])
unscaled_df.head()


,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Age,Sex_LabelEncoded
0,M,91,73,19,102.8,44.9,20.2,30.0,15,16.5,2
1,M,70,53,18,45.1,19.9,9.7,14.0,7,8.5,2
2,F,106,84,27,135.4,51.3,28.3,42.0,9,10.5,0
3,M,88,73,25,103.2,43.1,22.8,31.0,10,11.5,2
4,I,66,51,16,41.0,17.9,7.9,11.0,7,8.5,1


In [4]:
# Binary encode the categorical "Sex" column
unscaled_df = pd.get_dummies(unscaled_df, prefix=['Sex'], columns=['Sex'])
unscaled_df.head()

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Age,Sex_LabelEncoded,Sex_F,Sex_I,Sex_M
0,91,73,19,102.8,44.9,20.2,30.0,15,16.5,2,0,0,1
1,70,53,18,45.1,19.9,9.7,14.0,7,8.5,2,0,0,1
2,106,84,27,135.4,51.3,28.3,42.0,9,10.5,0,1,0,0
3,88,73,25,103.2,43.1,22.8,31.0,10,11.5,2,0,0,1
4,66,51,16,41.0,17.9,7.9,11.0,7,8.5,1,0,1,0


In [5]:
# Reorganize columns
column_order = ["Sex_LabelEncoded", "Sex_M", "Sex_F", "Sex_I", "Length", 
                "Diameter", "Height", "Whole_weight", "Shucked_weight", 
                "Viscera_weight", "Shell_weight", "Rings", "Age"]
unscaled_df = unscaled_df.reindex(columns=column_order)
unscaled_df.head()

,Sex_LabelEncoded,Sex_M,Sex_F,Sex_I,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Age
0,2,1,0,0,91,73,19,102.8,44.9,20.2,30.0,15,16.5
1,2,1,0,0,70,53,18,45.1,19.9,9.7,14.0,7,8.5
2,0,0,1,0,106,84,27,135.4,51.3,28.3,42.0,9,10.5
3,2,1,0,0,88,73,25,103.2,43.1,22.8,31.0,10,11.5
4,1,0,0,1,66,51,16,41.0,17.9,7.9,11.0,7,8.5


## Determine X and y (Example using Sex_LabelEncoded and y=Rings)

In [6]:
#For the neural network classification, the categories must have one-hot encoding, not just label encoding
#That is why only the one-hot encoded columns for sex were included and not the other versions

# Determine X 
X = unscaled_df[['Sex_M', 'Sex_F', 'Sex_I', 'Length', 'Diameter', 'Height', 'Whole_weight', 
                 'Shucked_weight', 'Viscera_weight', 'Shell_weight']]
X

,Sex_M,Sex_F,Sex_I,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight
0,1,0,0,91,73,19,102.8,44.9,20.2,30.0
1,1,0,0,70,53,18,45.1,19.9,9.7,14.0
2,0,1,0,106,84,27,135.4,51.3,28.3,42.0
3,1,0,0,88,73,25,103.2,43.1,22.8,31.0
4,0,0,1,66,51,16,41.0,17.9,7.9,11.0
...,...,...,...,...,...,...,...,...,...,...
4172,0,1,0,113,90,33,177.4,74.0,47.8,49.8
4173,1,0,0,118,88,27,193.2,87.8,42.9,52.1
4174,1,0,0,120,95,41,235.2,105.1,57.5,61.6
4175,0,1,0,125,97,30,218.9,106.2,52.2,59.2


## Checking Data Shape

In [7]:
# Determine y
y = unscaled_df["Rings"]
y

0       15
1        7
2        9
3       10
4        7
        ..
4172    11
4173    10
4174     9
4175    10
4176    12
Name: Rings, Length: 4177, dtype: int64

In [8]:
X.shape

(4177, 10)

In [9]:
y.shape

(4177,)

## Perform Train-Test-Split

In [10]:
# Must reshape y to have the correct vector to instead be a two-dimensional numpy array - matrix format
# This is needed for the other functions to use it correctly

y = y.values.reshape(-1,1)

In [11]:
y.shape

(4177, 1)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

## Scale Data and Evaluate


In [13]:
# Scale everything, and then unscale prediction to plot, but for the evaluation does not need to be unscaled unless RMSE

X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

y_scaler = StandardScaler().fit(y_train)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [14]:
# Deep Neural Network layout from https://towardsdatascience.com/deep-neural-networks-for-regression-problems-81321897ca33, but then simplified
# Model Set Up
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, input_dim = X_train_scaled.shape[1], activation='relu'))

#This is a simple neural network because it only contains one hidden layer between the input and output layers
# The Hidden Layers :
NN_model.add(Dense(256, activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam')
NN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1408      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 34,689
Trainable params: 34,689
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Training the model - Run

NN_model.fit(X_train_scaled, y_train_scaled, epochs=1000, batch_size=32, validation_split = 0.2)

Epoch 1/1000
79/79 [==============================] - 0s 3ms/step - loss: 0.5085 - val_loss: 0.5039
Epoch 2/1000
79/79 [==============================] - 0s 1ms/step - loss: 0.4905 - val_loss: 0.4724
Epoch 3/1000
79/79 [==============================] - 0s 1ms/step - loss: 0.4739 - val_loss: 0.4646
Epoch 4/1000
79/79 [==============================] - 0s 1ms/step - loss: 0.4626 - val_loss: 0.5020
Epoch 5/1000
79/79 [==============================] - 0s 1ms/step - loss: 0.4631 - val_loss: 0.4700
Epoch 6/1000
79/79 [==============================] - 0s 1ms/step - loss: 0.4569 - val_loss: 0.4564
Epoch 7/1000
79/79 [==============================] - 0s 1ms/step - loss: 0.4565 - val_loss: 0.4598
Epoch 8/1000
79/79 [==============================] - 0s 1ms/step - loss: 0.4563 - val_loss: 0.4591
Epoch 9/1000
79/79 [==============================] - 0s 1ms/step - loss: 0.4547 - val_loss: 0.4503
Epoch 10/1000
79/79 [==============================] - 0s 1ms/step - loss: 0.4494 - val_loss: 0.4530

In [16]:
predictions = NN_model.predict(X_test_scaled)
predictions.shape

(1045, 1)

In [17]:
model_loss = NN_model.evaluate(
    X_test_scaled, y_test_scaled, verbose=2)

33/33 - 0s - loss: 0.5587


In [18]:
r2 = r2_score(y_test_scaled, predictions)

In [19]:
mse = mean_squared_error(y_test_scaled, predictions)

In [20]:
print(f"Loss: {model_loss}")
print(f"R-squared: {r2}")
print(f"Mean Squared Error: {mse}")

Loss: 0.5586873888969421
R-squared: 0.3791133830758764
Mean Squared Error: 0.6118471458056424
